In [ ]:
import pandas as pd
import json
from datetime import datetime
from tqdm import tqdm

# Старые данные

In [ ]:
!wget -q -O archive_5.zip "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1BPKo4THVA1fIC27QoMwBoddZ4mA5vTzG"
!unzip -q archive_5.zip

In [ ]:
!pip install mysql-connector-python

     |████████████████████████████████| 37.6 MB 1.2 MB/s 


In [ ]:
df = pd.read_csv("archive_5.zip")

In [ ]:
# Очистка от NaN
df = df[df['VesselName'].notna()]
df = df[df['Length'].notna()]
df = df[df['Width'].notna()]
df = df[df['Status'].notna()]
df = df[df['Draft'].notna()]
df = df[df['IMO'].notna()]
df = df[df['CallSign'].notna()]
df = df[df['Cargo'].notna()]
df = df.loc[df['Width'] != 0]
df = df.loc[df['Length'] != 0]

In [ ]:
df = df.sort_values(by=['MMSI', 'BaseDateTime'])

In [ ]:
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
13361,123456789,2019-01-01T00:01:54,29.13867,-90.21792,0.0,102.6,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
23410,123456789,2019-01-01T00:03:54,29.13868,-90.21789,0.0,107.6,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
419366,123456789,2019-01-01T01:29:15,29.13866,-90.21792,0.0,170.6,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
457044,123456789,2019-01-01T01:38:55,29.13864,-90.21791,0.0,171.9,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
497602,123456789,2019-01-01T01:47:35,29.13864,-90.21793,0.0,130.9,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6906947,999999999,2019-01-01T22:08:51,36.95994,-76.32927,0.0,-180.5,511.0,US GOV VESSEL 51,IMO9999999,ZCDJ6,60.0,0.0,175.0,20.0,0.0,35.0,B
6926004,999999999,2019-01-01T22:12:02,36.95997,-76.32921,0.0,38.8,511.0,US GOV VESSEL 51,IMO9999999,ZCDJ6,60.0,0.0,175.0,20.0,0.0,35.0,B
6946555,999999999,2019-01-01T22:16:22,36.96008,-76.32914,0.0,164.0,511.0,US GOV VESSEL 51,IMO9999999,ZCDJ6,60.0,0.0,175.0,20.0,0.0,35.0,B
6958539,999999999,2019-01-01T22:18:32,36.96003,-76.32917,0.0,86.6,511.0,US GOV VESSEL 51,IMO9999999,ZCDJ6,60.0,0.0,175.0,20.0,0.0,35.0,B


In [ ]:
import mysql.connector as ms
connection = ms.connect(
        host="8.tcp.ngrok.io",
        port=10927,
        user="root",
        password="root",
        autocommit=True
    )
cursor = connection.cursor()
cursor.execute("SELECT @@VERSION;")
print(cursor.fetchone())

('10.4.12-MariaDB-log',)


In [ ]:
for index, i in tqdm(df.iterrows(), total=df.shape[0]):
  MMSI = int(i['MMSI'])
  dt = datetime.fromisoformat(i['BaseDateTime']).strftime('%Y-%m-%d %H:%M:%S')
  lat = float(i['LAT'])
  lon = float(i['LON'])
  cursor.execute('INSERT INTO `dpp.shlyopa.db`.stamps VALUES (%s, %s, %s, %s)', [MMSI, dt, lat, lon])
  connection.commit()
  # print(MMSI, dt, lat, lon)
  # break

  0%|          | 1035/972757 [10:26<163:29:10,  1.65it/s]


KeyboardInterrupt: ignored

In [ ]:
col = df['MMSI']
col = col.drop_duplicates()
col

13361      123456789
391753     205387000
2173       205460000
1092       205559000
11055      209251000
             ...    
1376334    720605000
100        827396670
9772       866998080
4088500    905961938
9970       999999999
Name: MMSI, Length: 1491, dtype: int64

In [ ]:
s = set()
for i in col:
  s.add(i)

In [ ]:
s = sorted(s)

In [ ]:
total = []
for i in s:
  vessel = df.loc[df['MMSI'] == int(i)]
  v = {
    'MMSI': int(df['MMSI'].to_list()[0]),
    'VesselName': str(df['VesselName'].to_list()[0]),
    'Length': float(df['Length'].to_list()[0]),
    'Width': float(df['Width'].to_list()[0]),
    'Points': []
  }
  for index, row in vessel.iterrows():
    v['Points'].append({
        'BaseDateTime': str(row['BaseDateTime']),
        'LAT': float(row['LAT']),
        'LON': float(row['LON'])
    })
  total.append(v)

In [ ]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [ ]:
!pip install mysql-connector-python

     |████████████████████████████████| 37.6 MB 1.3 MB/s 


In [ ]:
s

In [ ]:
total = []
count = 0
cursor.execute("TRUNCATE TABLE `dpp.shlyopa.db`.vessels;")
for i in s:
  count += 1
  vessel = df.loc[df['MMSI'] == int(i)]
  v = {
    'MMSI': int(vessel['MMSI'].to_list()[0]),
    'VesselName': str(vessel['VesselName'].to_list()[0]),
    'Length': float(vessel['Length'].to_list()[0]),
    'Width': float(vessel['Width'].to_list()[0]),
    'CallSign': str(vessel['CallSign'].to_list()[0]),
    'Cargo': int(vessel['Cargo'].to_list()[0]),
    'VesselType': int(vessel['VesselType'].to_list()[0]),
    'TranscieverClass': str(vessel['TranscieverClass'].to_list()[0])
  }
  cursor.execute("INSERT INTO `dpp.shlyopa.db`.vessels VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", [v['MMSI'], v['VesselName'], v['CallSign'], v['Length'], v['Width'], v['Cargo'], v['VesselType'], v['TranscieverClass']])
  connection.commit()
  printProgressBar(count, len(s))

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [ ]:
vessel

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TranscieverClass
13361,123456789,2019-01-01T00:01:54,29.13867,-90.21792,0.0,102.6,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
23410,123456789,2019-01-01T00:03:54,29.13868,-90.21789,0.0,107.6,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
419366,123456789,2019-01-01T01:29:15,29.13866,-90.21792,0.0,170.6,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
457044,123456789,2019-01-01T01:38:55,29.13864,-90.21791,0.0,171.9,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
497602,123456789,2019-01-01T01:47:35,29.13864,-90.21793,0.0,130.9,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7301527,123456789,2019-01-01T23:21:55,29.13868,-90.21791,0.0,107.5,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
7315580,123456789,2019-01-01T23:23:33,29.13866,-90.21792,0.0,112.2,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
7465671,123456789,2019-01-01T23:49:54,29.13866,-90.21792,0.0,130.4,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B
7489423,123456789,2019-01-01T23:56:15,29.13865,-90.21791,0.0,108.7,511.0,ODYSSEY,IMO0000000,ODYSSEY,37.0,0.0,25.0,10.0,4.5,90.0,B


In [ ]:
for index, row in vessel.iterrows():
  v['Points'].append({
      'BaseDateTime': row['BaseDateTime'],
      'LAT': row['LAT'],
      'LON': row['LON']
  })

In [ ]:
v

In [ ]:
df.to_csv("new.csv")

In [ ]:
!cp new.csv drive/MyDrive

In [ ]:
print(total[-1])

NameError: ignored

In [ ]:
import json
js = json.dumps(total)
with open('res.json', 'w') as f:
  f.write(js)

In [ ]:
!cp res.json drive/MyDrive

# Новые данные

In [ ]:
!wget "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_20.zip"
!wget "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_21.zip"
!wget "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_22.zip"
!wget "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_23.zip"
!wget "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_24.zip"
!wget "https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_25.zip"

--2022-04-06 13:44:18--  https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_20.zip
Resolving coast.noaa.gov (coast.noaa.gov)... 40.70.244.79
Connecting to coast.noaa.gov (coast.noaa.gov)|40.70.244.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328890017 (314M) [application/zip]
Saving to: ‘AIS_2021_07_20.zip.1’

AIS_2021_07_20.zip.   3%[                    ]  11.58M  3.35MB/s    eta 91s    ^C
--2022-04-06 13:44:22--  https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_21.zip
Resolving coast.noaa.gov (coast.noaa.gov)... 40.70.244.79
Connecting to coast.noaa.gov (coast.noaa.gov)|40.70.244.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 333240040 (318M) [application/zip]
Saving to: ‘AIS_2021_07_21.zip.1’

AIS_2021_07_21.zip.   5%[>                   ]  17.02M  3.83MB/s    eta 94s    ^C
--2022-04-06 13:44:29--  https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2021/AIS_2021_07_22.zip
Resolving coast.n

In [ ]:
dfs = [pd.read_csv(f"AIS_2021_07_{i}.zip") for i in tqdm(range(20, 26))]

100%|██████████| 6/6 [02:47<00:00, 27.98s/it]


In [ ]:
df = None
def clear():
  global df
  df = df[df['VesselName'].notna()]
  df = df[df['Length'].notna()]
  df = df[df['Width'].notna()]
  df = df[df['Status'].notna()]
  df = df[df['Draft'].notna()]
  df = df[df['IMO'].notna()]
  df = df[df['CallSign'].notna()]
  df = df[df['Cargo'].notna()]
  df = df.loc[df['Width'] != 0]
  df = df.loc[df['Length'] != 0]

In [ ]:
df = pd.read_csv(f"AIS_2021_07_20.zip")
clear()
for i in range(21, 26):
  df = pd.concat([df, pd.read_csv(f"AIS_2021_07_{i}.zip")])
  clear()

In [ ]:
df = df.sort_values(by=['MMSI', 'BaseDateTime'])

In [ ]:
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
1973256,11,2021-07-21T05:09:49,27.29221,-90.96786,0.0,181.6,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
2868604,11,2021-07-22T08:24:57,27.29227,-90.96789,0.0,273.1,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
4070700,11,2021-07-22T09:57:56,27.29227,-90.96795,0.0,221.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3643959,11,2021-07-22T10:28:01,27.29224,-90.96787,0.0,306.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3600756,11,2021-07-22T10:48:57,27.29234,-90.96787,0.0,250.3,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885014,999999999,2021-07-23T19:07:32,36.95449,-76.32848,0.0,24.0,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
7896142,999999999,2021-07-23T19:09:42,36.95443,-76.32858,0.1,7.8,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
6768473,999999999,2021-07-23T19:11:51,36.95438,-76.32864,0.0,108.6,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
9300349,999999999,2021-07-23T19:14:02,36.95445,-76.32861,0.1,24.9,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A


In [ ]:
df.to_csv("data.csv")

NameError: ignored

In [ ]:
!cp data.csv drive/MyDrive

In [ ]:
vs = df.copy()
vs

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
1973256,11,2021-07-21T05:09:49,27.29221,-90.96786,0.0,181.6,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
2868604,11,2021-07-22T08:24:57,27.29227,-90.96789,0.0,273.1,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
4070700,11,2021-07-22T09:57:56,27.29227,-90.96795,0.0,221.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3643959,11,2021-07-22T10:28:01,27.29224,-90.96787,0.0,306.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3600756,11,2021-07-22T10:48:57,27.29234,-90.96787,0.0,250.3,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885014,999999999,2021-07-23T19:07:32,36.95449,-76.32848,0.0,24.0,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
7896142,999999999,2021-07-23T19:09:42,36.95443,-76.32858,0.1,7.8,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
6768473,999999999,2021-07-23T19:11:51,36.95438,-76.32864,0.0,108.6,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
9300349,999999999,2021-07-23T19:14:02,36.95445,-76.32861,0.1,24.9,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A


In [ ]:
!cp drive/MyDrive/data.csv .

In [ ]:
df = pd.read_csv("data.csv", index_col=0)

In [ ]:
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
1973256,11,2021-07-21T05:09:49,27.29221,-90.96786,0.0,181.6,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
2868604,11,2021-07-22T08:24:57,27.29227,-90.96789,0.0,273.1,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
4070700,11,2021-07-22T09:57:56,27.29227,-90.96795,0.0,221.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3643959,11,2021-07-22T10:28:01,27.29224,-90.96787,0.0,306.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3600756,11,2021-07-22T10:48:57,27.29234,-90.96787,0.0,250.3,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885014,999999999,2021-07-23T19:07:32,36.95449,-76.32848,0.0,24.0,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
7896142,999999999,2021-07-23T19:09:42,36.95443,-76.32858,0.1,7.8,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
6768473,999999999,2021-07-23T19:11:51,36.95438,-76.32864,0.0,108.6,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
9300349,999999999,2021-07-23T19:14:02,36.95445,-76.32861,0.1,24.9,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A


In [ ]:
with open('save_data.sql', 'w') as f:
  for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    from datetime import datetime
    dt = datetime.fromisoformat(row['BaseDateTime']).strftime('%Y-%m-%d %H:%M:%S')
    f.write(f"INSERT INTO `dpp.shlyopa.db`.stamps(MMSI, datetime, latitude, longitude) VALUES ({row['MMSI']}, '{dt}', {row['LAT']}, {row['LON']});\n")

100%|██████████| 11718764/11718764 [15:37<00:00, 12497.76it/s]


In [ ]:
!cp save_data.sql drive/MyDrive

# vessels

In [ ]:
df = pd.read_csv("data.csv", index_col=0)
df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
1973256,11,2021-07-21T05:09:49,27.29221,-90.96786,0.0,181.6,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
2868604,11,2021-07-22T08:24:57,27.29227,-90.96789,0.0,273.1,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
4070700,11,2021-07-22T09:57:56,27.29227,-90.96795,0.0,221.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3643959,11,2021-07-22T10:28:01,27.29224,-90.96787,0.0,306.8,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
3600756,11,2021-07-22T10:48:57,27.29234,-90.96787,0.0,250.3,511.0,CONSTITUTION,IMO0000007,GC 680,90.0,5.0,53.0,38.0,0.0,90.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7885014,999999999,2021-07-23T19:07:32,36.95449,-76.32848,0.0,24.0,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
7896142,999999999,2021-07-23T19:09:42,36.95443,-76.32858,0.1,7.8,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
6768473,999999999,2021-07-23T19:11:51,36.95438,-76.32864,0.0,108.6,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A
9300349,999999999,2021-07-23T19:14:02,36.95445,-76.32861,0.1,24.9,511.0,T/T BOUDACIOUS,IMO0000000,TTTTTTT,37.0,8.0,2.0,2.0,0.0,90.0,A


In [ ]:
df = df.drop('BaseDateTime', 1)
df = df.drop('LAT', 1)
df = df.drop('LON', 1)
df = df.drop('SOG', 1)
df = df.drop('COG', 1)
df = df.drop('Heading', 1)
df = df.drop('IMO', 1)
df = df.drop('Status', 1)
df = df.drop('Draft', 1)
df = df.drop('TransceiverClass', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipyker

In [ ]:
df

,MMSI,VesselName,CallSign,VesselType,Length,Width,Cargo
1973256,11,CONSTITUTION,GC 680,90.0,53.0,38.0,90.0
2868604,11,CONSTITUTION,GC 680,90.0,53.0,38.0,90.0
4070700,11,CONSTITUTION,GC 680,90.0,53.0,38.0,90.0
3643959,11,CONSTITUTION,GC 680,90.0,53.0,38.0,90.0
3600756,11,CONSTITUTION,GC 680,90.0,53.0,38.0,90.0
...,...,...,...,...,...,...,...
7885014,999999999,T/T BOUDACIOUS,TTTTTTT,37.0,2.0,2.0,90.0
7896142,999999999,T/T BOUDACIOUS,TTTTTTT,37.0,2.0,2.0,90.0
6768473,999999999,T/T BOUDACIOUS,TTTTTTT,37.0,2.0,2.0,90.0
9300349,999999999,T/T BOUDACIOUS,TTTTTTT,37.0,2.0,2.0,90.0


In [ ]:
df.drop_duplicates(keep='last', inplace=True, subset=['MMSI'])

In [ ]:
df

,MMSI,VesselName,CallSign,VesselType,Length,Width,Cargo
7252020,11,CONSTITUTION,GC 680,90.0,53.0,38.0,90.0
7032020,111,BOOMVANG,EB643,90.0,47.0,35.0,90.0
7186859,112,NANSEN,EB602,90.0,47.0,35.0,90.0
6113324,507027,WEEKS 551,WBX3592,30.0,54.0,20.0,30.0
7294793,1193046,NAUTICAST,D11233,50.0,220.0,43.0,36.0
...,...,...,...,...,...,...,...
4696248,677029600,EQUALITY,5IM396,89.0,228.0,32.0,89.0
6828448,677031100,EL-SHADDAI,5IM411,70.0,34.0,8.0,0.0
6835323,677062300,PEPIN EXPRESS,5IM723,70.0,90.0,13.0,70.0
6516277,983542778,CARNIVAL MIRACLE TB6,H3VS,99.0,11.0,5.0,99.0


In [ ]:
with open('save_vessels.sql', 'w') as f:
  for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    f.write(f'INSERT INTO `dpp.shlyopa.db`.vessels VALUES ({row["MMSI"]}, "{row["VesselName"]}", "{row["CallSign"]}", {row["VesselType"]}, {row["Length"]}, {row["Width"]}, {row["Cargo"]});\n')

100%|██████████| 4192/4192 [00:00<00:00, 12261.99it/s]


In [ ]:
!cp save_vessels.sql drive/MyDrive/dpp